In [2]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np

In [3]:
import datajoint as dj
import pathlib
import itertools
import re
import tqdm

In [4]:
schema = dj.schema('photonics')
schema.spawn_missing_classes()

Connecting dimitri@localhost:3306


In [5]:
@schema
class Design(dj.Lookup):
    definition = """
    design : smallint     # design number
    ---
    design_title : varchar(255)    
    design_description : varchar(1000)
    design_path    : varchar(255)  
    geometry_file  : varchar(255)
    center_offset  : blob   # offset from legacy implementation
    efields : blob   # efield selection
    dfields : blob   # dfield selection
    """
    
    contents = [
        (1, '8-emitter-design', '', 'Design1/matrix_8pix_random_1200x1200x400_15-04-17', 
         'geometry.csv', (600, 600, 0), 
         (0,), (0,)),
        (3, "Wesley Sacher's shaped fields", "", 
         "Design3/matrix_wesley1_revised_revised_1000x1000x1000_15-12-15", 
         "geometry_beams_as_emitters_wesley1_1000x1000x1000.csv", (550, 510, 0), 
         (10, 11, 12, 13, 14, 15, 16, 17, 18), (1,)),
        (4, "Shaped fields with 30-degree-collection cones", 
         "50 emitters per shank, 30-degree emission detection fields", 
         "Design4/matrix_steer_and_collect_a1_b3_v3_16-06-02", 
         "steer_coll_a1_b3_beams_as_emitters_geometry.csv", (550, 510, 0), 
         (10, 11, 12, 13, 14, 15, 16, 17, 18), (2,)),
    ]

In [7]:
DSim()

dsim,dsim_description,detector_type choice in simulation,detector_width (um) along x-axis,detector_height (um) along y-axis,anisotropy factor in the Henyey-Greenstein formula,absorption_length (um) average travel path before a absoprtion event,scatter_length (um) average travel path before a scatter event,volume_dimx (voxels),volume_dimy (voxels),volume_dimz (voxels),pitch (um) spatial sampling period of the model volume
0,100% Efficient Lambertian 10x50 rect,one-sided,10.00,50.00,0.88,14000.0,100.0,1000,1000,1000,2.2
1,100% Efficient Lambertian 10x10 rect,one-sided,10.00,10.00,0.88,14000.0,100.0,1000,1000,1000,2.2
2,"Narrow selective as 4th power of cosine, 10x10 rect",narrowed,10.00,10.00,0.88,14000.0,100.0,1000,1000,1000,2.2


In [8]:
ESim * EField

esim,esim_description,beam_compression,y_steer the steer angle in the plane of the shank,emitter_width (um) along x-axis,emitter_height (um) along y-axis,anisotropy factor in the Henyey-Greenstein formula,absorption_length (um) average travel path before a absoprtion event,scatter_length (um) average travel path before a scatter event,volume_dimx (voxels),volume_dimy (voxels),volume_dimz (voxels),pitch (um) spatial sampling period of the model volume,volume probability of a photon emitted at given point getting picked up by the given detector,total_photons
0,Lambertian 10 x 10,1.0,0.0,10.00,10.00,0.88,14000.0,100.0,1000,1000,1000,2.2,=BLOB=,32541433
10,"Narrowed to pi/4, steered -24/64*pi",0.25,-1.1781,10.00,10.00,0.88,14000.0,100.0,1000,1000,1000,2.2,=BLOB=,32545338
11,"Narrowed to \pi/4, steered -18/64*pi",0.25,-0.883573,10.00,10.00,0.88,14000.0,100.0,1000,1000,1000,2.2,=BLOB=,32549329
12,"Narrowed to \pi/4, steered -12/64*pi",0.25,-0.589049,10.00,10.00,0.88,14000.0,100.0,1000,1000,1000,2.2,=BLOB=,32541420
13,"Narrowed to \pi/4, steered -6/64*pi",0.25,-0.294524,10.00,10.00,0.88,14000.0,100.0,1000,1000,1000,2.2,=BLOB=,32544172
14,"Narrowed to \pi/4, steered 0",0.25,0.0,10.00,10.00,0.88,14000.0,100.0,1000,1000,1000,2.2,=BLOB=,32545576
15,"Narrowed to \pi/4, steered +6/64*pi",0.25,0.294524,10.00,10.00,0.88,14000.0,100.0,1000,1000,1000,2.2,=BLOB=,32547334
16,"Narrowed to \pi/4, steered +12/64*pi",0.25,0.589049,10.00,10.00,0.88,14000.0,100.0,1000,1000,1000,2.2,=BLOB=,32539534
17,"Narrowed to \pi/4, steered +18/64*pi",0.25,0.883573,10.00,10.00,0.88,14000.0,100.0,1000,1000,1000,2.2,=BLOB=,32543131
18,"Narrowed to \pi/4, steered +24/64*pi",0.25,1.1781,10.00,10.00,0.88,14000.0,100.0,1000,1000,1000,2.2,=BLOB=,32544548


In [9]:
@schema
class Geometry(dj.Imported):
    definition = """
    -> Design
    ---
    """
    
    class Emitter(dj.Part):
        definition = """  # subtable of Geometry
            -> master
            emitter :smallint
            ----
            -> EField
            e_center_x   :float  # um
            e_center_y   :float  # um
            e_center_z   :float  # um
            e_norm_x : float 
            e_norm_y : float 
            e_norm_z : float 
            e_top_x : float 
            e_top_y : float 
            e_top_z : float 
            e_height : float  # um
            e_width  : float  # um
            e_thick  : float  # um            
            """
        
    class Detector(dj.Part):
        definition = """    # subtable of Geometry
            -> master
            detector :smallint
            ----
            -> DField
            d_center_x   :float  # um
            d_center_y   :float  # um
            d_center_z   :float  # um
            d_norm_x : float 
            d_norm_y : float 
            d_norm_z : float 
            d_top_x : float 
            d_top_y : float 
            d_top_z : float 
            d_height : float  # um
            d_width  : float  # um
            d_thick  : float  # um            
            """

    def make(self, key):
        self.insert1(key)
        legacy_filepath = '../legacy/matrices'
        detector_pattern = re.compile(r'Detector,"\((?P<center>.*)\)","\((?P<normal>.*)\)","\((?P<top>.*)\)",(?P<height>.*),(?P<width>.*),(?P<thick>.*)')
        emitter_pattern = re.compile(r'Emitter,"\((?P<center>.*)\)","\((?P<normal>.*)\)","\((?P<top>.*)\)",(?P<height>.*),(?P<width>.*),(?P<thick>.*)')
        d_count = itertools.count()
        e_count = itertools.count()
        efields, dfields = (Design & key).fetch1('efields', 'dfields')
        last_rec = {}
        origin = (Design & key).fetch1('center_offset')
        for line in pathlib.Path(legacy_filepath, *(Design & key).fetch1('design_path', 'geometry_file')).open():
            # detectors
            match = detector_pattern.match(line)
            if match:
                rec = dict(key, detector=next(d_count))
                rec.update(zip(('d_center_x','d_center_y','d_center_z'), 
                               (float(i)-offset for i, offset in zip(match['center'].split(','), origin))))
                if key['design'] == 1:
                    rec.update(zip(('d_norm_x','d_norm_y','d_norm_z'), 
                                   (float(i) for i in match['normal'].split(','))))
                    rec.update(zip(('d_top_x','d_top_y','d_top_z'), 
                                   (float(i) for i in match['top'].split(','))))
                else:
                    azimuth = (rec['d_center_z'] - 5)*np.pi*3/40 + np.pi/16
                    rec.update(d_norm_x=np.cos(azimuth), 
                               d_norm_y=np.sin(azimuth), 
                               d_norm_z=0,
                               d_top_x=0,
                               d_top_y=0,
                               d_top_z=1)
                rec.update(
                    d_height=float(match['height']), 
                    d_width=float(match['width']),
                    d_thick=float(match['thick']))
                if rec != last_rec:
                    self.Detector.insert(dict(rec, dsim=dfield, detector=next(d_count)) 
                                for dfield in dfields)
                    last_rec = rec
                continue
                
            # emitters
            match = emitter_pattern.match(line)
            if match:
                rec = dict(key)
                rec.update(zip(('e_center_x','e_center_y','e_center_z'), 
                               (float(i)-offset for i, offset in zip(match['center'].split(','), origin))))
                if key['design'] == 1:
                    rec.update(zip(('e_norm_x','e_norm_y','e_norm_z'), 
                                   (float(i) for i in match['normal'].split(','))))
                    rec.update(zip(('e_top_x','e_top_y','e_top_z'), 
                                   (float(i) for i in match['top'].split(','))))
                else:
                    azimuth = (rec['e_center_z'] - 5)*np.pi*3/40 + np.pi/16
                    rec.update(e_norm_x=np.cos(azimuth), 
                               e_norm_y=np.sin(azimuth), 
                               e_norm_z=0,
                               e_top_x=0,
                               e_top_y=0,
                               e_top_z=1)

                rec.update(
                    e_height=float(match['height']), 
                    e_width=float(match['width']),
                    e_thick=float(match['thick']))
                if rec != last_rec:
                    self.Emitter.insert(
                        dict(rec, esim=efield, emitter=next(e_count)) 
                        for efield in efields)
                    last_rec = rec
                continue

In [11]:
Geometry.populate(display_progress=True)

100%|██████████| 3/3 [01:02<00:00, 20.72s/it]


In [12]:
Geometry.Emitter() & 'design=1'

design design number,emitter,esim,e_center_x um,e_center_y um,e_center_z um,e_norm_x,e_norm_y,e_norm_z,e_top_x,e_top_y,e_top_z,e_height um,e_width um,e_thick um
1,0,0,14.5,-195.0,200.0,0.92388,0.382683,0.0,0.0,0.0,1.0,10.0,10.0,2.0
1,1,0,14.5,-195.0,150.0,0.92388,0.382683,0.0,0.0,0.0,1.0,10.0,10.0,2.0
1,2,0,14.5,-195.0,100.0,0.92388,0.382683,0.0,0.0,0.0,1.0,10.0,10.0,2.0
1,3,0,14.5,-195.0,50.0,0.92388,0.382683,0.0,0.0,0.0,1.0,10.0,10.0,2.0
1,4,0,14.5,-195.0,0.0,0.92388,0.382683,0.0,0.0,0.0,1.0,10.0,10.0,2.0
1,5,0,14.5,-205.0,200.0,0.92388,-0.382683,0.0,0.0,0.0,1.0,10.0,10.0,2.0
1,6,0,14.5,-205.0,150.0,0.92388,-0.382683,0.0,0.0,0.0,1.0,10.0,10.0,2.0
1,7,0,14.5,-205.0,100.0,0.92388,-0.382683,0.0,0.0,0.0,1.0,10.0,10.0,2.0
1,8,0,14.5,-205.0,50.0,0.92388,-0.382683,0.0,0.0,0.0,1.0,10.0,10.0,2.0
1,9,0,14.5,-205.0,0.0,0.92388,-0.382683,0.0,0.0,0.0,1.0,10.0,10.0,2.0


In [24]:
set(zip(*(Geometry.Emitter & 'design=1').fetch('e_norm_x', 'e_norm_y', 'e_norm_z')))

{(-0.92388, -0.382683, -0.0),
 (-0.92388, 0.382683, 0.0),
 (-0.382683, -0.92388, -0.0),
 (-0.382683, 0.92388, 0.0),
 (0.382683, -0.92388, 0.0),
 (0.382683, 0.92388, 0.0),
 (0.92388, -0.382683, 0.0),
 (0.92388, 0.382683, 0.0)}

# Design 1 Geometry

In [ ]:
r = {'design': 1}
fig, axx = plt.subplots(1, 2, figsize=(8,4))

ax = axx[0]

ax.set_aspect('equal')
ax.set_xlabel(r'$x\; (\mu\mathsf{m})$', fontsize=10)
ax.set_ylabel(r'$y\; (\mu\mathsf{m})$', fontsize=10)
ax.set_xticks(np.r_[-600:601:200])
ax.set_yticks(np.r_[-600:601:200])
ax.grid(True)

_ = (Geometry.Emitter & 'design=1').fetch('e_center_x', 'e_center_y', 'e_norm_x', 'e_norm_y')
for x, y, nx, ny in zip(*_):
    ax.plot((x, x+30*nx), (y, y+30*ny), 'g', alpha=0.2, lw=0.5)

_ = (Geometry.Detector & 'design=1').fetch('d_center_x', 'd_center_y', 'd_norm_x', 'd_norm_y')
for x, y, nx, ny in zip(*_):
    ax.plot((x, x+30*nx), (y, y+30*ny), 'k', alpha=0.2)


ax = axx[1]

ax.set_aspect('equal')
ax.set_xlabel(r'$y\; (\mu\mathsf{m})$', fontsize=10)
ax.set_ylabel(r'$z\; (\mu\mathsf{m})$', fontsize=10)
ax.grid(True)

_ = (Geometry.Emitter & 'design=1').fetch('e_center_x', 'e_center_z', 'e_norm_x', 'e_norm_z')
for x, z, nx, nz in zip(*_):
    ax.plot((x, x+30*nx), (z, z+30*nz), 'g', lw=0.5)

_ = (Geometry.Detector & 'design=1').fetch('d_center_y', 'd_center_z', 'd_norm_y', 'd_norm_z')
for y, z, ny, nz in zip(*_):
    ax.plot((y, y+30*ny), (z, z+30*nz), 'k', lw=1)

fig.suptitle('Design {design}'.format(**r))


fig.savefig('geometry-1.pdf')

# Designs 3 and 4 
The orientations of pixels (norms) were missing. 

25 shanks with 34 steerable emitters and 33 detectors

In [ ]:
r = {'design': 4}
fig, axx = plt.subplots(1, 2, figsize=(8,4))

ax = axx[0]

ax.set_aspect('equal')
ax.set_xlabel(r'$x\; (\mu\mathsf{m})$', fontsize=10)
ax.set_ylabel(r'$y\; (\mu\mathsf{m})$', fontsize=10)
ax.set_xticks(np.r_[-600:601:200])
ax.set_yticks(np.r_[-600:601:200])
ax.grid(True)

_ = (Geometry.Emitter & r).fetch('e_center_x', 'e_center_y', 'e_norm_x', 'e_norm_y')
for x, y, nx, ny in tqdm.tqdm(zip(*_)):
    ax.plot((x, x+30*nx), (y, y+30*ny), 'g-', alpha=0.2, lw=1)

_ = (Geometry.Detector & r).fetch('d_center_x', 'd_center_y', 'd_norm_x', 'd_norm_y')
_ = set(zip(*_))
for x, y, nx, ny in tqdm.tqdm(_):
    ax.plot((x, x+30*nx), (y, y+30*ny), 'k-', alpha=0.2)

ax = axx[1]

ax.set_aspect('equal')
ax.set_xlabel(r'$y\; (\mu\mathsf{m})$', fontsize=10)
ax.set_ylabel(r'$z\; (\mu\mathsf{m})$', fontsize=10)
ax.grid(True)

_ = (Geometry.Emitter & r).fetch('e_center_y', 'e_center_z', 'e_norm_y', 'e_norm_z')
_ = set(zip(*_))
for y, z, ny, nz in tqdm.tqdm(_):
    ax.plot((y, y+30*ny), (z, z+30*nz), 'g-', lw=1)

_ = (Geometry.Detector & r).fetch('d_center_y', 'd_center_z', 'd_norm_y', 'd_norm_z')
_ = set(zip(*_))
for y, z, ny, nz in tqdm.tqdm(_):
    ax.plot((y, y+30*ny), (z, z+30*nz), 'k-', lw=1)

fig.suptitle('Design {design}'.format(**r))

fig.savefig('geometry-4.png', dpi=300)

In [ ]:
set(zip(*(Geometry.Emitter & 'design=3').fetch('e_norm_x', 'e_norm_y', 'e_norm_z')))

In [ ]:
set(zip(*(Geometry.Detector & 'design=3').fetch('d_norm_x', 'd_norm_y', 'd_norm_z')))

In [ ]:
z = (Geometry.Detector & 'design=3').fetch('d_center_z')
plt.scatter(np.cos((z - 5)*np.pi*3/40 + np.pi/16), np.sin((z - 5)*np.pi*3/40 + np.pi/16))
plt.axis('equal');

In [ ]:
!open geometry-4.pdf

In [ ]:
len(set(zip(*(Geometry.Emitter & r).fetch('e_center_x', 'e_center_y', 'e_center_z'))))

In [ ]:
len(set(zip(*(Geometry.Detector & r).fetch('d_center_x', 'd_center_y', 'd_center_z'))))

In [ ]:
825/25

In [ ]:
set(zip(*(Geometry.Detector & 'design=1').fetch('d_top_x', 'd_top_y', 'd_top_z')))

In [ ]:
set(zip(*(Geometry.Detector & 'design=1').fetch('d_norm_x', 'd_norm_y', 'd_norm_z')))

In [ ]:
a = np.array(sorted(list(set((Geometry.Emitter & 'design=3').fetch('e_center_z')))))

In [ ]:
np.diff(a)

In [ ]:
schema.spawn_missing_classes()
dj.Diagram(schema)

In [ ]:
Geometry.Detector()

In [ ]:
sorted(np.r_[:135*8:135] % 360)

In [ ]:
Geometry.Detector & 'design=1'

In [ ]:
Geometry.Detector

In [ ]:
dj.Diagram(schema)

In [ ]:
Geometry()

In [ ]:
DField * DSim

In [ ]:
Geometry.drop()

In [ ]:
dj.Diagram(schema)

In [ ]:
Design()

In [ ]:
np.r_[22.5:1080:135] % 360

In [ ]:
360*3

In [ ]:
min(np.array(sorted(list(set((Geometry.Emitter & 'design=4').fetch('e_center_z'))))))

In [ ]:
ls

In [ ]:
!open geometry-4.png